# Georgian Wine Expert: Model Inference and Comparison

This notebook demonstrates the fine-tuned Georgian wine specialist model compared to the base Flan-T5 model.

## 1. Setup

In [ ]:
!pip install transformers torch --quiet

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Load Models

In [ ]:
# Load base Flan-T5 model
print("Loading base Flan-T5 model...")
base_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Load fine-tuned model
print("Loading fine-tuned Georgian wine model...")
expert_tokenizer = AutoTokenizer.from_pretrained("./georgian-wine-model")
expert_model = AutoModelForSeq2SeqLM.from_pretrained("./georgian-wine-model")

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
base_model.to(device)
expert_model.to(device)

print(f"\n✅ Models loaded on device: {device}")

## 3. Define Query Function (Required)

In [ ]:
def ask_expert(question: str, model_path: str) -> str:
    """
    Takes a question about Georgian wine and returns a knowledgeable response.
    
    Args:
        question: A question about Georgian winemaking
        model_path: Path to the fine-tuned model directory
    
    Returns:
        Model's response as a string
    """
    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    
    # Format input
    input_text = f"Answer this question about Georgian wine: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate response
    outputs = model.generate(
        **inputs,
        max_length=256,
        num_beams=4,
        early_stopping=True
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

print("✅ ask_expert() function defined")

## 4. Helper Function for Side-by-Side Comparison

In [ ]:
def compare_models(question: str):
    """
    Compare base Flan-T5 response with fine-tuned expert model.
    """
    # Format input
    input_text = f"Answer this question about Georgian wine: {question}"
    
    # Base model response
    base_inputs = base_tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
    base_inputs = {k: v.to(device) for k, v in base_inputs.items()}
    
    base_outputs = base_model.generate(
        **base_inputs,
        max_length=256,
        num_beams=4,
        early_stopping=True
    )
    base_response = base_tokenizer.decode(base_outputs[0], skip_special_tokens=True)
    
    # Expert model response
    expert_inputs = expert_tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
    expert_inputs = {k: v.to(device) for k, v in expert_inputs.items()}
    
    expert_outputs = expert_model.generate(
        **expert_inputs,
        max_length=256,
        num_beams=4,
        early_stopping=True
    )
    expert_response = expert_tokenizer.decode(expert_outputs[0], skip_special_tokens=True)
    
    # Display comparison
    print("="*100)
    print(f"QUESTION: {question}")
    print("="*100)
    print("\n🔵 BASE FLAN-T5 (No fine-tuning):")
    print("-"*100)
    print(base_response)
    print("\n🟢 FINE-TUNED GEORGIAN WINE EXPERT:")
    print("-"*100)
    print(expert_response)
    print("\n" + "="*100 + "\n")

print("✅ compare_models() function defined")

## 5. Side-by-Side Comparison: Base vs Fine-Tuned

In [ ]:
# Test Question 1
compare_models("What is a qvevri?")

In [ ]:
# Test Question 2
compare_models("How old is Georgian winemaking tradition?")

In [ ]:
# Test Question 3
compare_models("What is Saperavi and where is it grown?")

In [ ]:
# Test Question 4
compare_models("What is the difference between Kakhetian and Imeretian winemaking styles?")

In [ ]:
# Test Question 5
compare_models("Why does Georgian white wine sometimes look orange?")

In [ ]:
# Test Question 6
compare_models("How is a qvevri cleaned and maintained?")

In [ ]:
# Test Question 7
compare_models("When was Georgian winemaking added to UNESCO's heritage list?")

In [ ]:
# Test Question 8
compare_models("Why are qvevri buried underground?")

In [ ]:
# Test Question 9
compare_models("What is Rkatsiteli grape known for?")

In [ ]:
# Test Question 10
compare_models("What foods pair well with Georgian amber wine?")

In [ ]:
# Test Question 11
compare_models("What is the Kakheti wine region known for?")

In [ ]:
# Test Question 12
compare_models("How did Georgian winemaking change during the Soviet era?")

## 6. Test the ask_expert() Function

In [ ]:
# Example usage of the ask_expert function
question = "What is skin contact fermentation?"
answer = ask_expert(question, "./georgian-wine-model")

print(f"Question: {question}")
print(f"\nExpert Answer:\n{answer}")

## 7. Interactive Testing

In [ ]:
# Try your own question
custom_question = "What is Mtsvane?"  # Change this to any question
compare_models(custom_question)

## Summary

This notebook demonstrated:

✅ Loading both base and fine-tuned models  
✅ Implementing the `ask_expert()` function as required  
✅ Side-by-side comparison showing the fine-tuned model's superior domain knowledge  
✅ Interactive testing capability  

**Key Observations:**
- The base Flan-T5 model gives generic or incorrect answers
- The fine-tuned model provides accurate, detailed, domain-specific responses
- Fine-tuning successfully transferred specialized knowledge to the model